# Know yourself, Know your logs aka Baselining
BTV Project Obsidian, 2023

Author: juju43, https://blueteamvillage.org/programs/project-obsidian/ https://discord.gg/blueteamvillage
<img align="right" width="100" height="100" src="https://cfc.blueteamvillage.org/media/call-for-content-2021/img/20200622_BTVillage_logos_RGB_pos_hcOC7Qx.png">

![BTV Know yourself Know your logs title slide](BTV-Know_yourself_logs.png)

## FIXME!
* `TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NotImplementedError'` in /usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:168 + msticpy 2.5.2/2.5.0
* 2.4.0 deprecation warning for some queries `/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'`

## Agenda

* Why understanding your environment is Critical
* Key Questions
* Obsidian 2023 logs
  * What is in the environment?
  * What applications?
  * What matters?

## Introduction
* At time of EDR, SIEM and logs may seem too complex and costly.
* I believe it is still part of the permanent journey to protect an environment.
* Most EDR are blind if exploitation at only application level (ex: MoveIt) or for PaaS (simply not existing).
* Applications logs are usually not standardized and complex.
* Many options: SIEM, data lake, buckets/storage...

## Why understanding your environment is Critical
* And why it often goes through logs among other things, but not only.
* Also documentation... which often is not loved.

<div align="center" style="font-weight: bold">
Relentlessy documenting your environment is critical.<br />
Share knowledge and ensure no single point of failure.
</div>


## Key questions
* What happens in my environment
* What's normal
* What's abnormal
* How I would know

## Obsidian 2023 logs

### Code initialization

In [1]:
# Check we are running Python 3.6
import sys

MIN_REQ_PYTHON = (3, 6)
if sys.version_info < MIN_REQ_PYTHON:
    print("Check the Kernel->Change Kernel menu and ensure that Python 3.6")
    print("or later is selected as the active kernel.")
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

In [2]:
# Imports
import pandas as pd
import msticpy.nbtools as nbtools
from datetime import datetime, timedelta
import os

In [3]:
# path to config file
os.environ["MSTICPYCONFIG"] = "/home/ubuntu/msticpyconfig.yaml"
from msticpy.nbtools import *
from msticpy.data.data_providers import QueryProvider
from msticpy.common.wsconfig import WorkspaceConfig
from msticpy.nbtools.data_viewer import DataViewer
from msticpy.vis.matrix_plot import plot_matrix
from msticpy.nbtools import process_tree as ptree
from msticpy.context.geoip import GeoLiteLookup, IPStackLookup

print("Imports Complete")

Imports Complete


In [4]:
# Interactive settings edit
# https://msticpy.readthedocs.io/en/latest/getting_started/SettingsEditor.html#using-mpconfigfile-to-check-and-manage-your-msticpyconfig-yaml
from msticpy.config import MpConfigFile, MpConfigEdit, MpConfigControls

mpconfig = MpConfigFile()
# mpconfig.load_default()
# mpconfig.view_settings()
mpconfig

In [5]:
# q_times = nbwidgets.QueryTime(units='hours', max_before=72, before=1, max_after=0)
q_times = nbwidgets.QueryTime(
    origin_time=datetime(2023, 4, 29), units="days", max_before=3, before=1, max_after=0
)

q_times.display()

In [6]:
# Ensure to use appropriate filters
query_common_args = '''earliest="04/29/2023:01:00:00" latest="04/29/2023:23:00:00"'''
results_limit = 10

In [7]:
query_common_args = query_common_args.strip()

In [8]:
# Configuration
# if free splunk,
#  * enable the 'allowRemoteLogin' setting in your server.conf file - /opt/splunk/etc/system/local/server.conf
qry_prov = QueryProvider("Splunk")
qry_prov.connect()

connected


In [9]:
# pandas
pd.set_option("display.max_colwidth", 500)

### What is in the environment?
* Linux
* Windows
* Others

In [10]:
query = f"""| tstats count WHERE ((index=* OR index=_*) {query_common_args}) BY index
"""
df_index = qry_prov.exec_query(query)
df_index.head(results_limit)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 97.00it/s]


,index,count
0,_audit,59357
1,_telemetry,9
2,linux,517912
3,main,258365
4,nginx,2
5,osquery,7728
6,sysmon,108442
7,sysmonforlinux,74236
8,windows,15413


In [11]:
query = f"""| tstats latest(_time) as last_seen, earliest(_time) as first_seen where (index=* {query_common_args}) by index host source sourcetype | convert ctime(last_seen), ctime(first_seen)
"""
df_timeperiod = qry_prov.exec_query(query)
# df_timeperiod.head(results_limit)
df_timeperiod

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 95.41it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,index,host,source,sourcetype,last_seen,first_seen
0,linux,ip-172-16-40-100,linux,filebeat,04/29/2023 22:59:52,04/29/2023 17:46:12
1,linux,ip-172-16-50-101,linux,filebeat,04/29/2023 22:34:12,04/29/2023 17:46:41
2,linux,ip-172-16-60-11,linux,filebeat,04/29/2023 22:34:34,04/29/2023 17:46:24
3,linux,ip-172-16-60-12,linux,filebeat,04/29/2023 22:34:24,04/29/2023 17:46:28
4,linux,ip-172-16-60-13,linux,filebeat,04/29/2023 22:34:33,04/29/2023 17:46:20
...,...,...,...,...,...,...
87,windows,wkst16.magnumtempus.financial,windows,winlogbeat,04/29/2023 22:55:29,04/29/2023 17:52:30
88,windows,wkst17.magnumtempus.financial,windows,winlogbeat,04/29/2023 22:34:42,04/29/2023 17:51:46
89,windows,wkst18.magnumtempus.financial,windows,winlogbeat,04/29/2023 22:34:46,04/29/2023 17:52:26
90,windows,wkst19.magnumtempus.financial,windows,winlogbeat,04/29/2023 22:34:47,04/29/2023 17:52:16


In [12]:
query = f"""| tstats count WHERE ((index=*) {query_common_args}) BY index, host
| stats values(index) as indexes by host
"""
df_assets = qry_prov.exec_query(query)
df_assets.head(results_limit)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 95.94it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,host,indexes
0,WKST13,windows
1,dc.magnumtempus.financial,"[sysmon, windows]"
2,hmail,"[sysmon, windows]"
3,hmialpha,sysmon
4,hmibeta,sysmon
5,hmicharlie,"[sysmon, windows]"
6,hmidelta,sysmon
7,iot-eng-wkst.magnumtempus.financial,"[sysmon, windows]"
8,iot-jumpbox,"[sysmon, windows]"
9,ip-172-16-22-10,main


### What applications?
* Linux ProcessName
* Windows Event Channel or Source

In [14]:
# process name extracted only for journald collection, not for log file with default filebeat. as such, results may be incomplete
query = f"""search index=linux {query_common_args}
| stats count by journald.process.name
"""
df_linux_apps = qry_prov.exec_query(query)
df_linux_apps.head(results_limit)

Waiting Splunk job to complete: 384.692it [00:06, 62.43it/s]                       
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,journald.process.name,count
0,(systemd),3
1,chronyd,7
2,cron,28
3,dbus-daemon,1
4,filebeat,4476
5,java,95
6,multipathd,5
7,openplc,230182
8,osqueryd,19
9,packagekitd,1


In [16]:
query = f"""search index=windows {query_common_args}
| stats count by winlog.provider_name,winlog.channel
"""
df_win_apps = qry_prov.exec_query(query)
df_win_apps.head(results_limit)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 96.95it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,winlog.provider_name,winlog.channel,count
0,Desktop Window Manager,Application,68
1,EventLog,System,1
2,LsaSrv,System,1
3,Microsoft-Windows-CAPI2,Application,6
4,Microsoft-Windows-DHCPv6-Client,System,2
5,Microsoft-Windows-DNS-Client,System,1
6,Microsoft-Windows-Dhcp-Client,System,4
7,Microsoft-Windows-DistributedCOM,System,64
8,Microsoft-Windows-DriverFrameworks-UserMode,System,2
9,Microsoft-Windows-Eventlog,Security,1


### Log Level?
* Some logs have a categorization
  * critical, error, warning, information, debug...
  * authentication, data dump/takeout/download, privileged/administrative activities, security...

In [19]:
query = f"""search index=windows log.level="error" {query_common_args}
| stats count by event.code,event.provider,message
| sort - count
"""
df_errors = qry_prov.exec_query(query)
df_errors.head(results_limit)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 93.75it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,event.code,event.provider,message,count
0,10010,Microsoft-Windows-DistributedCOM,The server {AB8902B4-09CA-4BB6-B78D-A8F59079A8D5} did not register with DCOM within the required timeout.,92
1,10010,Microsoft-Windows-DistributedCOM,The server {AAC1009F-AB33-48F9-9A21-7F5B88426A2E} did not register with DCOM within the required timeout.,20
2,5858,Microsoft-Windows-WMI-Activity,"Id = {00000000-0000-0000-0000-000000000000}; ClientMachine = WKST08; User = NT AUTHORITY\NETWORK SERVICE; ClientProcessId = 4072; Component = Unknown; Operation = Start IWbemServices::DeleteInstance - Root\Rsop : __Namespace.name=""NSS_1_5_21_2369732838_3797832421_459094119_1129""; ResultCode = 0x80041002; PossibleCause = Unknown",4
3,5858,Microsoft-Windows-WMI-Activity,Id = {00000000-0000-0000-0000-000000000000}; ClientMachine = IOT-ENG-WKST; User = MAGNUMTEMPUS\seth.morgan; ClientProcessId = 7924; Component = Unknown; Operation = Start IWbemServices::ExecQuery - ROOT\CIMV2 : SELECT HypervisorPresent FROM Win32_ComputerSystem; ResultCode = 0x80041032; PossibleCause = Unknown,2
4,5858,Microsoft-Windows-WMI-Activity,"Id = {00000000-0000-0000-0000-000000000000}; ClientMachine = WKST02; User = NT AUTHORITY\NETWORK SERVICE; ClientProcessId = 5000; Component = Unknown; Operation = Start IWbemServices::ExecQuery - ROOT\CIMV2 : SELECT Manufacturer, Product FROM Win32_BaseBoard; ResultCode = 0x80041032; PossibleCause = Unknown",2
5,5858,Microsoft-Windows-WMI-Activity,Id = {00000000-0000-0000-0000-000000000000}; ClientMachine = WKST08; User = NT AUTHORITY\SYSTEM; ClientProcessId = 6628; Component = Unknown; Operation = Start IWbemServices::ExecQuery - ROOT\CIMV2 : SELECT * FROM Win32_OptionalFeature WHERE name = 'NetFX3'; ResultCode = 0x80041032; PossibleCause = Unknown,2
6,5858,Microsoft-Windows-WMI-Activity,Id = {00000000-0000-0000-0000-000000000000}; ClientMachine = WKST08; User = NT AUTHORITY\SYSTEM; ClientProcessId = 6628; Component = Unknown; Operation = Start IWbemServices::ExecQuery - ROOT\CIMV2 : SELECT * FROM Win32_OptionalFeature WHERE name = 'WAS-NetFxEnvironment'; ResultCode = 0x80041032; PossibleCause = Unknown,2
7,5858,Microsoft-Windows-WMI-Activity,"Id = {00000000-0000-0000-0000-000000000000}; ClientMachine = WKST09; User = NT AUTHORITY\NETWORK SERVICE; ClientProcessId = 10996; Component = Unknown; Operation = Start IWbemServices::ExecQuery - ROOT\CIMV2 : SELECT Manufacturer, Product FROM Win32_BaseBoard; ResultCode = 0x80041032; PossibleCause = Unknown",2
8,5858,Microsoft-Windows-WMI-Activity,"Id = {00000000-0000-0000-0000-000000000000}; ClientMachine = WKST09; User = NT AUTHORITY\NETWORK SERVICE; ClientProcessId = 7524; Component = Unknown; Operation = Start IWbemServices::ExecQuery - ROOT\CIMV2 : SELECT Manufacturer, Product FROM Win32_BaseBoard; ResultCode = 0x80041032; PossibleCause = Unknown",2
9,5858,Microsoft-Windows-WMI-Activity,Id = {00000000-0000-0000-0000-000000000000}; ClientMachine = WKST10; User = MAGNUMTEMPUS\reese.juvo; ClientProcessId = 5436; Component = Unknown; Operation = Start IWbemServices::ExecQuery - ROOT\CIMV2 : SELECT HypervisorPresent FROM Win32_ComputerSystem; ResultCode = 0x80041032; PossibleCause = Unknown,2


### What matters?
* Sometimes people knows, sometimes not
* Providing view of what you see may help to identify things
* May identify other information
  * Non-security operational issues (disk full...)
  * Data that should be there - gap
  * Data that should not be there (low value, risky like passwords, tokens, PHI, PCI...)
  * Be selective. No point to collect logs without value.

### Share and Report!
* Written documentation
* Working controls
* Identified gaps, eventually with possible remediations

## That's all folks!
Thanks to
* [BTV project obsidian](https://www.blueteamvillage.org/programs/project-obsidian/) contributors
* Projects jupyter notebook, pandas, msticpy, OTRF / [Infosec Jupyterthon](https://infosecjupyterthon.com)

<h1 align="center" style="font-weight: bold;">Thank you</h1>

<div align="center">
Join the conversation<br />
https://discord.blueteamvillage.org<br />
Please Provide Feedback<br />
feedback-obsidian@blueteamvillage.org<br />
</div>

## References

* Logs
  * https://www.honeynet.org/challenges/forensic-challenge-5-2010-log-mysteries/
  * https://chuvakin.blogspot.com/2010/03/simple-log-review-checklist-released.html
  * https://www.sans.org/brochure/course/log-management-in-depth/6
  * Strategy 7: Select and Collect the Right Data, [11 Strategies of a World-Class Cybersecurity Operations Center, MITRE](https://www.mitre.org/sites/default/files/2022-04/11-strategies-of-a-world-class-cybersecurity-operations-center.pdf)
  * [LogLevel Enum - Microsoft](https://learn.microsoft.com/en-us/dotnet/api/microsoft.extensions.logging.loglevel?view=dotnet-plat-ext-7.0)
  * https://www.malwarearchaeology.com/cheat-sheets
  * https://what2log.com/
  * https://github.com/SigmaHQ/sigma
  * [MITRE Assistant - Stats by Datasources](https://docs-ma.vercel.app/docs/project/Data%20Samples/V9%20Enterprise/Stats_By_Datasources)
* Detect PII
  * https://microsoft.github.io/presidio/; https://github.com/microsoft/presidio
  * https://pypi.org/project/piicatcher/; https://github.com/tokern/piicatcher
  * https://learn.microsoft.com/en-us/azure/cognitive-services/language-service/personally-identifiable-information/how-to-call
  * https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/textanalytics/azure-ai-textanalytics/samples
  * https://lantern.splunk.com/Security/Use_Cases/Compliance/Detecting_Personally_Identifiable_Information_(PII)_in_log_data_for_GDPR_compliance
  * https://www.bindb.com/bin-database
  * https://www.paypalobjects.com/en_GB/vhelp/paypalmanager_help/credit_card_numbers.htm
  * https://en.wikipedia.org/wiki/Health_Level_7